In [ ]:
import uproot
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib as mpl

from util import *
import var
import cut
import data
import hist

import importlib

from pyanalib import panda_helpers

from scipy import optimize, stats

In [ ]:
plt.rcParams.update({'font.size': 16})
# mpl.rcParams['lines.linewidth'] = 4.

dosave = True
savedir = "plots_2_28_24/signalbox/"

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

In [ ]:
# configured
DETSYST = 15.426
DETSYST

In [ ]:
NUMISYST = 8.6

In [ ]:
# fname = filedir + "F-CohLike_nom_evt_spectrum_blah.df"
# fname = "/icarus/data/users/gputnam/DMCP2023G/mc-F/wdetsyst-chi2tgt/F-Nu_loose_evt_chi2tgt_spectrum_detsyst.df"
fname = "/icarus/data/users/gputnam/DMCP2023G/mc-F-spectra/F-Nu_evt_spectrum.df"

In [ ]:
cvdf = pd.read_hdf(fname, "s")

In [ ]:
bins = np.array([0.21, 0.225, 0.245, 0.265, 0.29, 0.32, 0.35, 0.4, 0.5])
centers = (bins[:-1] + bins[1:]) / 2
var = cvdf.mass

In [ ]:
modes = [
    cvdf.genie_mode != 3,
    cvdf.genie_mode == 3
]

when_all = cvdf.thnumi < 5

In [ ]:
# Track splitting correction and error

In [ ]:

_ = plt.hist(var[when_all], bins=bins, weights=(cvdf.weight/(cvdf.cathode_weight*cvdf.gap_weight))[when_all], 
             histtype="step", linewidth=2, label="Nominal")

_ = plt.hist(var[when_all], bins=bins, weights=(cvdf.weight/(cvdf.gap_weight))[when_all], histtype="step", linewidth=2,
            label="Cathode Split Corr.")

_ = plt.hist(var[when_all], bins=bins, weights=cvdf.weight[when_all], histtype="step", linewidth=2, 
            label="Cathode + Gap Split Corr.")

l1 = plt.legend(loc="lower right")
plt.ylabel("Events / 2.41e20 POT / bin")
plt.xlabel("Reconstructed Mass")

# get corrected total and uncertainty
N,_ = np.histogram(var[when_all], bins=bins, weights=cvdf.weight[when_all])
Nerr,_ = np.histogram(var[when_all], bins=bins, weights=cvdf.weight[when_all]*(1+np.sqrt(cvdf.cathode_weight_err**2 + cvdf.gap_weight_err**2)[when_all]))
err = np.abs(N - Nerr)
tracksplit_frac_err = err/N

unc = plt.fill_between(bins, np.append(N + err, 0), np.append(N - err, 0),
            edgecolor="gray", facecolor="none", step="post", hatch="//", linewidth=0, label="Uncertainty\n on Corr.")
plt.legend(handles=[unc], loc="upper left", frameon=False)
plt.gca().add_artist(l1)


if dosave:
    plt.savefig(savedir + "tracksplit_corr_w_unc.pdf", bbox_inches="tight")
    plt.savefig(savedir + "tracksplit_corr_w_unc.svg", bbox_inches="tight")

In [ ]:
# Total fractional error
100*err.sum() / N.sum()

In [ ]:
flux_uncs = []
g4_uncs = []
xsec_uncs = []
coh_uncs = []
det_uncs = []
mcstat_uncs = []
stat_uncs = []

for ibin, (mlo, mhi) in enumerate(zip(bins[:-1], bins[1:])):
    when = when_all & (var > mlo) & (var < mhi)
    
    flux_uncs.append(100*(cvdf[when][["wgt_flux_univ_%i" % i for i in range(100)]].multiply(cvdf.weight[when], 0)).sum().std()/cvdf.weight[when].sum())
    flux_uncs[-1] = np.sqrt(flux_uncs[-1]**2 + NUMISYST**2)
    g4_uncs.append(100*(cvdf[when][["wgt_g4_univ_%i" % i for i in range(100)]].multiply(cvdf.weight[when], 0)).sum().std()/cvdf.weight[when].sum())
    xsec_uncs.append(100*(cvdf[when][["wgt_xsec_univ_%i" % i for i in range(100)]].multiply(cvdf.weight[when], 0)).sum().std()/cvdf.weight[when].sum())
    coh_uncs.append(100*(cvdf[when][["wgt_coh_univ_%i" % i for i in range(100)]].multiply(cvdf.weight[when], 0)).sum().std()/cvdf.weight[when].sum())
    det_uncs.append(np.sqrt(DETSYST**2 + (100*tracksplit_frac_err[ibin])**2))
    mcstat_uncs.append(100*np.sqrt((cvdf.weight[when]**2).sum())/cvdf.weight[when].sum())
    stat_uncs.append(100/np.sqrt(cvdf.weight[when].sum()))
    
flux_uncs = np.array(flux_uncs)
g4_uncs = np.array(g4_uncs)
det_uncs = np.array(det_uncs)
xsec_uncs = np.array(xsec_uncs)
coh_uncs = np.array(coh_uncs)
mcstat_uncs = np.array(mcstat_uncs)
total_uncs = np.sqrt(flux_uncs**2 + g4_uncs**2 + det_uncs**2 + xsec_uncs**2 + coh_uncs**2 + mcstat_uncs**2)
stat_uncs = np.array(stat_uncs)

In [ ]:
_ = plt.hist(centers, bins=bins, weights=flux_uncs, histtype="step", label="Flux", linewidth=2)
_ = plt.hist(centers, bins=bins, weights=det_uncs, histtype="step", label="Detector", linewidth=2)
_ = plt.hist(centers, bins=bins, weights=g4_uncs, histtype="step", label="G4", linewidth=2)
_ = plt.hist(centers, bins=bins, weights=xsec_uncs, histtype="step", label="Incoh. xsec", linewidth=2)
_ = plt.hist(centers, bins=bins, weights=coh_uncs, histtype="step", label="Coh. xsec", linewidth=2)
_ = plt.hist(centers, bins=bins, weights=mcstat_uncs, histtype="step", label="M.C. Stat", linewidth=2)
_ = plt.hist(centers, bins=bins, weights=total_uncs, histtype="step", label="Total", linewidth=2.5, color="black")

plt.ylabel("Uncertainty [%]")
plt.xlabel("Reconstructed $m_{\mu\mu}$ [GeV/c$^2$]")
plt.legend(ncol=2)
plt.ylim([plt.ylim()[0], plt.ylim()[1]*1.6])

if dosave:
    plt.savefig(savedir + "signalbox_syst_neutrino.pdf", bbox_inches="tight")
    plt.savefig(savedir + "signalbox_syst_neutrino.svg", bbox_inches="tight")

In [ ]:
_ = plt.hist(centers, bins=bins, weights=flux_uncs, histtype="step", label="Flux", linewidth=2)
_ = plt.hist(centers, bins=bins, weights=det_uncs, histtype="step", label="Detector", linewidth=2)
_ = plt.hist(centers, bins=bins, weights=g4_uncs, histtype="step", label="G4", linewidth=2)
_ = plt.hist(centers, bins=bins, weights=xsec_uncs, histtype="step", label="Incoh. xsec", linewidth=2)
_ = plt.hist(centers, bins=bins, weights=coh_uncs, histtype="step", label="Coh. xsec", linewidth=2)
_ = plt.hist(centers, bins=bins, weights=mcstat_uncs, histtype="step", label="M.C. Stat", linewidth=2)
_ = plt.hist(centers, bins=bins, weights=stat_uncs, histtype="step", label="Stat.", linewidth=2.5, color="gray")

_ = plt.hist(centers, bins=bins, weights=total_uncs, histtype="step", label="Total Syst.", linewidth=2.5, color="black", linestyle="--")
_ = plt.hist(centers, bins=bins, weights=np.sqrt(stat_uncs**2 + total_uncs**2), histtype="step", label="Total Stat.\n+ Syst.", linewidth=2.5, color="black")

plt.ylabel("Uncertainty [%]")
plt.xlabel("Reconstructed $m_{\mu\mu}$ [GeV/c$^2$]")
plt.legend(ncol=2)
plt.ylim([plt.ylim()[0], plt.ylim()[1]*1.2])

if dosave:
    plt.savefig(savedir + "signalbox_syststat_neutrino.pdf", bbox_inches="tight")
    plt.savefig(savedir + "signalbox_syststat_neutrino.svg", bbox_inches="tight")